In [1]:
# dataset retrieved from https://archive.ics.uci.edu/dataset/352/online+retail
import pandas as pd
import numpy as np

In [4]:
file_path = "data/Online_Retail.csv"

df = pd.read_csv(file_path,
                 header= 0,
                 nrows= 5,
                 parse_dates= ["InvoiceDate"])

print(df.dtypes)

InvoiceNo               int64
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int64
Country                object
dtype: object
